In [1]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_aer.primitives import Sampler
from qiskit_aer.backends import AerSimulator
from qiskit.circuit.library import TwoLocal
#from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit import transpile, QuantumCircuit
from qiskit_optimization.translators import to_docplex_mp
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader
from qiskit_optimization.translators import from_docplex_mp

import networkx as nx
import jax
import jax.numpy as jnp

import optimizer as opt
import time

In [2]:
mod = Model()
mod = ModelReader.read_model('/Users/vaibhawkumar/vaibhaw/projects/algos/Client-Moderna-Optimization/lp_G0_lp_c/seq_1079_c.lp')

#convert linear program to qubo
qp = from_docplex_mp(mod)
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
quboOp, offset = qubo.to_ising()

Hp = {'Op': quboOp, 'offset': offset}

Q_info = qubo.objective.quadratic.coefficients.copy()
l_diag = qubo.objective.linear
for k in range(qp.get_num_vars()):
    Q_info[k,k] += l_diag[k]

Q = Q_info.tocoo()
Q_data = np.stack([Q.row, Q.col, Q.data], axis=1)
scaling = np.abs(np.max(Q))

n_qubits = Q.shape[0]

n_qubits

32

In [3]:
## we create a binary array representing the Hamiltonian, needed for IQPOpt

def find(s, ch):
    return tuple(i for i, ltr in enumerate(s) if ltr == ch)

ising_adj = {}
for op in Hp['Op'].label_iter():
    ising_adj[find(op[0], 'Z')] = op[1]
    
binary_arr = []
for key in ising_adj.keys():
    bin_str = n_qubits * [0] #a bit string with n_qubit zeros
    for item in key:
        bin_str[n_qubits-item-1] = 1 #to account for endian convention in qiskit
    binary_arr.append(bin_str)

## create IQP ansatz

In [4]:
G_hp_cube = nx.generators.hypercube_graph(n= int(np.log2(n_qubits))) #
G_hp_cube.edges()

def tuple_to_int(binary_tuple):

    binary_string = "".join(map(str, binary_tuple))  

    return int(binary_string, 2)

edge_list = []
for elem in G_hp_cube.edges():
    edge_list.append((tuple_to_int(elem[0]), tuple_to_int(elem[1])))

G_layout = nx.Graph()
G_layout.add_nodes_from(range(0, n_qubits))
for edge in edge_list:
    G_layout.add_edge(edge[0], edge[1])

#nx.draw(G_layout, with_labels=True, font_weight='bold')

import iqpopt as iqp
from iqpopt.utils import local_gates, nearest_neighbour_gates

gates = nearest_neighbour_gates(G_layout, distance=1, max_weight=2) 

circuit = iqp.IqpSimulator(n_qubits, gates)

## inputs to the optimizer

In [5]:
ops = jnp.array(binary_arr) #binary array representing operators

bin_init = np.random.choice([1, 0], size=n_qubits, p=[0.1, 0.9])
theta_initial = np.hstack([np.full(n_qubits, np.pi/8), np.zeros(len(circuit.gates)-n_qubits)])

#theta_initial = np.random.uniform(0, 2*np.pi, len(circuit.gates))

n_samples = 2 ** 13

In [6]:
len(theta_initial)

112

## create optimizer object

In [7]:
QuantumOptimizer = opt.Quantum_Optimizer()
QuantumOptimizer.set_optimizer(num_binary_var=n_qubits, circuit=circuit, n_samples=n_samples, ops=ops, 
                               coupling=ising_adj, offset=Hp['offset'])

## Run optimization 

In [8]:
st = time.time()

opt_run = QuantumOptimizer.run(
    qc_params_init = theta_initial, 
    max_epoch=3, 
    random_update=True, 
    intermediate_result_display=True)
print(time.time() - st)

epoch = 0 	 step = 0 	 expectation = 3133.6
epoch = 0 	 step = 1 	 expectation = 3133.6
epoch = 0 	 step = 2 	 expectation = 3133.6
epoch = 0 	 step = 3 	 expectation = 3133.6
epoch = 0 	 step = 4 	 expectation = 3133.6
epoch = 0 	 step = 5 	 expectation = 3133.6
epoch = 0 	 step = 6 	 expectation = 3114.9
epoch = 0 	 step = 7 	 expectation = 3114.9
epoch = 0 	 step = 8 	 expectation = 3114.9
epoch = 0 	 step = 9 	 expectation = 3070.3
epoch = 0 	 step = 10 	 expectation = 3070.3
epoch = 0 	 step = 11 	 expectation = 3070.3
epoch = 0 	 step = 12 	 expectation = 3070.3
epoch = 0 	 step = 13 	 expectation = 3070.3
epoch = 0 	 step = 14 	 expectation = 3070.3
epoch = 0 	 step = 15 	 expectation = 3070.3
epoch = 0 	 step = 16 	 expectation = 3070.3
epoch = 0 	 step = 17 	 expectation = 3070.3
epoch = 0 	 step = 18 	 expectation = 3070.3
epoch = 0 	 step = 19 	 expectation = 3070.3
epoch = 0 	 step = 20 	 expectation = 3070.3
epoch = 0 	 step = 21 	 expectation = 3070.3
epoch = 0 	 step = 2

## AI transpiler

In [9]:
opt_params = [float(i) for i in QuantumOptimizer.theta_log[len(QuantumOptimizer.theta_log)-1]]

qc = QuantumCircuit(n_qubits)
for i in range(0, n_qubits):
    qc.h(i)
    
for edge, param in zip(edge_list, opt_params):
    qc.rzz(param, edge[0], edge[1])
    
for i in range(0, n_qubits):
    qc.h(i)
    
qc.measure_all()

from qiskit.transpiler import PassManager
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_transpiler.ai.routing import AIRouting
 
ai_passmanager = PassManager([
  AIRouting(backend_name="ibm_kyiv", optimization_level=3, layout_mode="optimize", local_mode=True)
])

for i in range(0, 1):
    qc_trans = ai_passmanager.run(qc)
    print("total_depth:", qc_trans.depth(), "2q-depth:", qc_trans.depth(lambda x:x.operation.num_qubits==2))
    print(qc_trans.count_ops())

total_depth: 70 2q-depth: 61
OrderedDict([('swap', 160), ('rzz', 80), ('h', 64), ('measure', 32), ('barrier', 1)])


## MPS simulator

from qiskit_aer.primitives import Sampler
from qiskit_aer.backends import AerSimulator

shots = 2**15

sampler = Sampler(backend_options={'method' : 'matrix_product_state', 'n_qubits': n_qubits})
sampler.set_options(shots=shots)

prob_list = []
for i in range(0, 1):
    job = sampler.run(qc)

    result = job.result()
    quasi_dist = result.quasi_dists[0]
    prob_dict = quasi_dist.binary_probabilities()
    prob_list.append(prob_dict)
    bin_list = list(prob_dict.keys())  

import matplotlib.pyplot as plt

def energy(x):
    x = np.array(list(x)).astype(int)
    x = np.flip(x) 
    return x @ Q @ x

#loop over the prob dictionary
for dicts in prob_list:
    #convert bitstrings to energy for better plotting
    
    ee = {x:energy(x) for x in dicts.keys()}
    gs = min(ee, key=ee.get)
    counts = {x:p * shots for (x, p) in zip(dicts.keys(), dicts.values())}
    plt.plot(np.min(list(ee.values())), counts[gs], 'o')

plt.ylabel('counts')
plt.xlabel('energy')
plt.legend(['trial:' + str(j) for j in range(0, 1)])
#plt.savefig("count_variation.pdf", dpi=300)
plt.show()

## Hardware runs

In [10]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
 
mytoken = "b260e676440f704486b4d9a672b55a3635f0525f7ce93d8724d84313d84a09883cb871db46b16d9521d1d9c2d19fa993641380b480245d23a38a5a311f0efe9d"
#QiskitRuntimeService.save_account(channel="ibm_quantum", token=mytoken, overwrite=True, set_as_default=True)

service = QiskitRuntimeService(channel="ibm_quantum", token=mytoken, instance="modernatx/internal/ibm-collab")
#QiskitRuntimeService(instance="client-enablement/its/its-skill-buildi")
backend = service.backend('ibm_kyiv')
n_shots = 2 ** 18

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(qc)
 
sampler = Sampler(backend)
sampler.options.dynamical_decoupling.enable = True
sampler.options.twirling.enable_gates = False
sampler.options.twirling.num_randomizations = "auto"
sampler.options.execution.rep_delay = max(backend.rep_delay_range) #delay between subsequent circuits

job = sampler.run([isa_circuit], shots=n_shots)
result = job.result()
 
# Get results for the first (and only) PUB
pub_result = result[0]
 
print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

RuntimeJobFailureError: 'Unable to retrieve job result. ValueError: too many values to unpack (expected 2)'